In [1]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
import numba

In [2]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [3]:
arr = np.array([np.array(list(line), dtype = np.int64) for line in rawstring.splitlines()],dtype=np.int64)

In [4]:
arr.shape

(100, 100)

In [5]:
arr

array([[4, 3, 9, ..., 6, 6, 2],
       [1, 7, 1, ..., 8, 3, 6],
       [5, 6, 4, ..., 9, 6, 5],
       ...,
       [1, 7, 9, ..., 1, 8, 9],
       [3, 4, 2, ..., 9, 6, 8],
       [1, 9, 7, ..., 4, 5, 7]])

In [6]:
@numba.jit()
def calculate_costs():
    pos = [0,0]
    path = []
    min_cost = np.zeros_like(arr)
    visited = np.zeros_like(arr,dtype=bool)
    visited_n = np.zeros_like(arr,dtype=bool)

    visited[0,0] = True
    visited_n[0,0] = True

    updated_old = np.zeros_like(visited, dtype=bool)
    while not np.all(visited):
        print(np.sum(visited))
        print(np.sum(updated_old))
        updated = np.zeros_like(visited, dtype=bool)
        for idx in range(visited.shape[0]):
            for idy in range(visited.shape[1]):
                if idx == 0 and idy == 0:
                    continue
                elif (not np.all([visited[np.clip(x,0,arr.shape[0]-1),np.clip(y,0,arr.shape[1]-1)] for x,y in  [[idx,idy],[idx-1,idy],[idx+1,idy],[idx,idy-1],[idx,idy+1]]]) and np.any([visited[np.clip(x,0,arr.shape[0]-1),np.clip(y,0,arr.shape[1]-1)] for x,y in  [[idx-1,idy],[idx+1,idy],[idx,idy-1],[idx,idy+1]]])) or np.any([updated_old[np.clip(x,0,arr.shape[0]-1),np.clip(y,0,arr.shape[1]-1)] for x,y in  [[idx-1,idy],[idx+1,idy],[idx,idy-1],[idx,idy+1]]]):
                    visited_n[idx, idy] = True
                    new_cost = arr[idx, idy] + np.min([min_cost[np.clip(x,0,arr.shape[0]-1),np.clip(y,0,arr.shape[1]-1)] for x,y in  [[idx-1,idy],[idx+1,idy],[idx,idy-1],[idx,idy+1]] if visited[np.clip(x,0,arr.shape[0]-1),np.clip(y,0,arr.shape[1]-1)]])
                    if min_cost[idx, idy] != 0:
                        old_cost = min_cost[idx, idy]
                        min_cost[idx, idy] = min(new_cost, min_cost[idx, idy])
                        if new_cost < old_cost:
                            updated[idx, idy] = True
                    else:
                        min_cost[idx, idy] = new_cost
                        updated[idx, idy] = True
        updated_old = copy.deepcopy(updated)
        visited = copy.deepcopy(visited_n)

    return min_cost

#calculate_costs()

In [6]:
arr_2 = np.hstack([arr+i for i in range(5)])
arr_2 = np.vstack([arr_2+i for i in range(5)])
arr_2[arr_2>9] -= 9

In [7]:
def calculate_costs2(arr):
    padded_x = np.pad(arr,pad_width=1, mode='constant', constant_values = 0)
    views = np.lib.stride_tricks.sliding_window_view(
                    padded_x, (3,3)
                )
                
    min_cost = np.zeros_like(arr)
    visited = np.zeros_like(arr,dtype=bool)
    visited_n = np.zeros_like(arr,dtype=bool)

    padded_v = np.pad(visited,pad_width=1, mode='constant', constant_values = False)
    v_views = np.lib.stride_tricks.sliding_window_view(
                    padded_v, (3,3)
                )

    padded_v[1,1] = True
    visited_n[0,0] = True

    while not np.all(padded_v[1:-1,1:-1]):
        print(np.sum(padded_v))
        #print(np.sum(updated_old))
        #updated = np.zeros_like(visited, dtype=bool)
        padded_mc = np.pad(min_cost,pad_width=1, mode='constant', constant_values = -10)
        mc_views = np.lib.stride_tricks.sliding_window_view(
                    padded_mc, (3,3)
                )
        for idx in range(views.shape[0]):
            for idy in range(views.shape[1]):
                view = views[idx,idy,:]
                mc_view = mc_views[idx,idy]
                v_view = v_views[idx,idy]
                if idx == 0 and idy == 0:
                    continue
                if not v_view[1,1] and np.any(v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]):
                    padded_mc[idx+1,idy+1] = view[1,1]+np.min(mc_view[[1-1,1+1,1,1],[1,1,1+1,1-1]][v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]])
                    padded_v[idx+1,idy+1] = True
        
        padded_mc_o = copy.deepcopy(padded_mc)
        first = True
        while not np.all(np.equal(padded_mc_o,padded_mc)) or first:
            print(np.sum(np.not_equal(padded_mc_o,padded_mc)))
            padded_mc_o = copy.deepcopy(padded_mc)
            first = False
            for idx in range(views.shape[0]):
                for idy in range(views.shape[1]):
                    view = views[idx,idy,:]
                    mc_view = mc_views[idx,idy]
                    v_view = v_views[idx,idy]
                    if idx == 0 and idy == 0:
                        continue
                    else:
                        padded_mc[idx+1,idy+1] = view[1,1]+np.min(mc_view[[1-1,1+1,1,1],[1,1,1+1,1-1]][v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]])
                        padded_v[idx+1,idy+1] = True
            

    return padded_mc[1:-1,1:-1]

calculate_costs2(arr_2)

1
0
243061
239616
192637
216265
173032
122264
130500
120524
112135
104482
103963
82129
69311
66154
58315
57438
50136
46895
41779
39213
35193
34549
31535
29668
27561
25672
23982
21073
21036
18410
17975
15889
14802
13731
12722
10915
10005
8917
8197
7020
6328
5806
5216
5006
4529
4090
3576
3189
2773
2438
2056
1688
1450
1304
1161
1043
900
767
681
591
494
456
472
425
277
162
47
30
15
15
9


array([[   0,    3,   12, ..., 1693, 1694, 1700],
       [   1,    8,    9, ..., 1692, 1699, 1700],
       [   6,   12,   13, ..., 1691, 1692, 1701],
       ...,
       [1732, 1727, 1725, ..., 2916, 2923, 2931],
       [1739, 1735, 1731, ..., 2920, 2921, 2928],
       [1742, 1737, 1733, ..., 2912, 2916, 2922]])

In [19]:
@numba.jit(parallel=True)
def calculate_costs3(arr):
    padded_x = np.pad(arr,pad_width=1, mode='constant', constant_values = 0)
    views = np.lib.stride_tricks.sliding_window_view(
                    padded_x, (3,3)
                )
                
    min_cost = np.zeros_like(arr)
    visited = np.zeros_like(arr,dtype=bool)
    visited_n = np.zeros_like(arr,dtype=bool)

    padded_v = np.pad(visited,pad_width=1, mode='constant', constant_values = False)
    v_views = np.lib.stride_tricks.sliding_window_view(
                    padded_v, (3,3)
                )

    padded_v[1,1] = True
    visited_n[0,0] = True

    updated = np.zeros_like(padded_v,dtype=bool)
    u_views = np.lib.stride_tricks.sliding_window_view(
                    updated, (3,3)
                )
    
    update_count = 0

    padded_mc = np.pad(min_cost,pad_width=1, mode='constant', constant_values = -10)
    mc_views = np.lib.stride_tricks.sliding_window_view(
                    padded_mc, (3,3)
                )
    updates = 1
    while not np.all(padded_v[1:-1,1:-1]) or updates:
        print(f"new iteration, all visited: {np.all(padded_v[1:-1,1:-1])}, updated elements: {update_count}")
        if np.all(padded_v[1:-1,1:-1]):
            idxs, idys = np.where(updated)
            updated = np.zeros_like(padded_v,dtype=bool)
            for idx_u, idy_u in zip(idxs, idys):
                for idx in [idx_u+1,idx_u-1]:
                    if idx < 0 or idx >= arr.shape[0]:
                        continue
                    for idy in [idy_u+1, idy_u-1]:
                        if idy < 0 or idy >= arr.shape[1]:
                            continue
                        view = views[idx,idy,:]
                        oldval = padded_mc[idx+1,idy+1]
                        mc_view = mc_views[idx,idy]
                        v_view = v_views[idx,idy]
                        u_view = u_views[idx,idy]
                        newval = view[1,1]+np.min(mc_view[[1-1,1+1,1,1],[1,1,1+1,1-1]][v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]])
                        if newval < oldval:
                            padded_mc[idx+1,idy+1] = newval
                            updated[idx+1,idy+1] = True
        else:
            updated = np.zeros_like(padded_v,dtype=bool)
            for idx in range(views.shape[0]):
                for idy in range(views.shape[1]):
                    view = views[idx,idy,:]
                    mc_view = mc_views[idx,idy]
                    v_view = v_views[idx,idy]
                    u_view = u_views[idx,idy]
                    if idx == 0 and idy == 0:
                        continue
                    if (not v_view[1,1] and np.any(v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]])) or np.any(u_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]):
                        oldval = padded_mc[idx+1,idy+1]
                        newval = view[1,1]+np.min(mc_view[[1-1,1+1,1,1],[1,1,1+1,1-1]][v_view[[1-1,1+1,1,1],[1,1,1+1,1-1]]])
                        padded_mc[idx+1,idy+1] = newval
                        updated[idx+1,idy+1] = True
                        padded_v[idx+1,idy+1] = True
        updates = np.any(updated)
        update_count = np.sum(updated)
        
    
    return padded_mc[1:-1,1:-1]

calculate_costs3(arr_2)

/tmp/ipykernel_21610/3119334806.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculate_costs3" failed type inference due to: Use of unsupported NumPy function 'numpy.pad' or unsupported use of the function.

File "../../../tmp/ipykernel_21610/3119334806.py", line 3:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_21610/3119334806.py (3)

File "../../../tmp/ipykernel_21610/3119334806.py", line 3:
<source missing, REPL/exec in use?>

  @numba.jit(parallel=True)
/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "calculate_costs3" was compiled in object mode without forceobj=True.

File "../../../tmp/ipykernel_21610/3119334806.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/opt/conda/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the 

new iteration, all visited: False, updated elements: 0
new iteration, all visited: True, updated elements: 249999
new iteration, all visited: True, updated elements: 247116
new iteration, all visited: True, updated elements: 162451
new iteration, all visited: True, updated elements: 84714
new iteration, all visited: True, updated elements: 73363
new iteration, all visited: True, updated elements: 71355
new iteration, all visited: True, updated elements: 68337
new iteration, all visited: True, updated elements: 46828
new iteration, all visited: True, updated elements: 39574
new iteration, all visited: True, updated elements: 33320
new iteration, all visited: True, updated elements: 26429
new iteration, all visited: True, updated elements: 21982
new iteration, all visited: True, updated elements: 19341
new iteration, all visited: True, updated elements: 17097
new iteration, all visited: True, updated elements: 14433
new iteration, all visited: True, updated elements: 12828
new iteration,

array([[   0,    3,   12, ..., 1693, 1694, 1700],
       [   1,    8,    9, ..., 1692, 1699, 1700],
       [   6,   12,   13, ..., 1691, 1692, 1701],
       ...,
       [1733, 1728, 1726, ..., 2916, 2923, 2931],
       [1740, 1736, 1732, ..., 2920, 2921, 2928],
       [1743, 1738, 1734, ..., 2912, 2916, 2922]])

In [7]:
n = deque()
n.append(1)
n.append(2)
print(n.popleft())

1


In [14]:
min_cost

array([[   0, 1061, 1070, ...,    0,    0,    0],
       [   1, 1058, 1071, ...,    0,    0,    0],
       [   6, 1051, 1075, ...,    0,    0,    0],
       ...,
       [ 542,  566, 1659, ...,    0,    0,    0],
       [ 545,  559, 1661, ...,    0,    0,    0],
       [ 546,  555, 1668, ...,    0,    0,    0]])

In [ ]:
move

Error: Session cannot generate requests

In [15]:
import sys

In [12]:
sys.getrecursionlimit()

3000

In [16]:
sys.setrecursionlimit(100_000)